In [1]:
from pathlib import Path

import numpy as np
import pandas as pd
import requests_cache
import wbgapi

requests_cache.install_cache()

route = Path.home() / "OneDrive" / "Rawdata"

In [2]:
countrycode = pd.read_excel(
    route / "Country Code" / "Countrycode.xlsx",
    sheet_name="Sheet1",
    na_values="..",
)
code_alpha3 = countrycode[["Numeric", "Alpha-3 code"]]
code_wb = countrycode[["Numeric", "WBCode"]]
code_CN = countrycode[["Numeric", "CountryName_CN"]]

In [3]:
# WDI_World Development Index
wb = (
    pd.read_csv(
        route / "World Bank World Development Index" / "WDI_csv" / "WDIData.csv",
        na_values="..",
    )
    .drop(columns=["Unnamed: 67"])
    .rename(columns={str(year): year for year in range(1996, 2023)})
)
reshaped_wb = (
    wb.drop(columns=["Country Name", "Indicator Name"])
    .melt(id_vars=["Country Code", "Indicator Code"])
    .rename(columns={"variable": "year"})
    .pivot_table(
        columns="Indicator Code",
        values="value",
        index=["Country Code", "year"],
    )
)
WDI = reshaped_wb.loc[
    (list(set(code_alpha3["Alpha-3 code"]) - {"ESH", "REU", "MYT"}), range(1996, 2022)),
    :,
].sort_index()
WDI.index.rename(["Alpha-3 code", "Year"], inplace=True)

WDI = (
    WDI.reset_index()
    .rename(
        columns={
            row["Indicator Code"]: row["Indicator Name"]
            for row in wb[["Indicator Code", "Indicator Name"]]
            .drop_duplicates()
            .to_dict("records")
        }
    )
    .merge(code_alpha3, how="left", on="Alpha-3 code")
    .drop(columns=["Alpha-3 code"])
)

In [4]:
cols = []
WDI_2021 = WDI.query("Year == 2021")
for col in WDI:
    if not WDI_2021[col].isnull().all():
        cols.append(col)
WDI_2021[cols]

,Year,Land under cereal production (hectares),Cereal production (metric tons),Crop production index (2014-2016 = 100),Food production index (2014-2016 = 100),Livestock production index (2014-2016 = 100),Cereal yield (kg per hectare),Trade in services (% of GDP),"Communications, computer, etc. (% of service imports, BoP)","Primary income payments (BoP, current US$)",...,"Voice and Accountability: Percentile Rank, Upper Bound of 90% Confidence Interval",Voice and Accountability: Standard Error,Battle-related deaths (number of people),"Internally displaced persons, new displacement associated with conflict and violence (number of cases)","Internally displaced persons, new displacement associated with disasters (number of cases)","Internally displaced persons, total displaced by conflict and violence (number of people)","Intentional homicides, female (per 100,000 female)","Intentional homicides, male (per 100,000 male)","Intentional homicides (per 100,000 people)",Numeric
25,2021,3052269.0,3053114.70,115.48,114.59,108.68,1000.3,10.875888,46.258240,6.138745e+09,...,29.951691,0.120518,NaN,NaN,22000.0,NaN,NaN,NaN,NaN,24
51,2021,309009.0,457790.83,121.22,123.70,128.06,1481.5,NaN,NaN,NaN,...,16.908213,0.129830,24.0,94.0,87000.0,19000.0,NaN,NaN,NaN,108
77,2021,1639128.0,2308912.73,129.16,119.17,107.47,1408.6,8.154368,26.367513,4.923823e+08,...,45.410629,0.123860,NaN,NaN,10000.0,2700.0,NaN,NaN,NaN,204
103,2021,3761051.0,4710191.10,113.09,116.21,113.90,1252.4,9.987538,32.701381,1.256550e+09,...,49.758453,0.120698,1006.0,682000.0,NaN,1580000.0,NaN,NaN,NaN,854
129,2021,121752.0,122201.31,147.33,106.04,99.51,1003.7,9.221913,61.723281,1.684915e+08,...,66.183578,0.120854,NaN,NaN,NaN,NaN,7.629581,NaN,10.546962,72
155,2021,178596.0,142808.96,114.03,111.26,109.20,799.6,NaN,NaN,NaN,...,21.256039,0.130674,928.0,487000.0,24000.0,692000.0,NaN,NaN,NaN,140
181,2021,1288329.0,2955486.12,126.06,122.03,117.08,2294.0,6.731567,28.522164,2.780337e+09,...,36.231884,0.120698,NaN,NaN,NaN,302000.0,NaN,NaN,NaN,384
207,2021,2273123.0,3794655.70,104.70,103.47,100.67,1669.4,8.691039,21.705653,8.730686e+08,...,21.739130,0.120698,581.0,131000.0,1800.0,909000.0,NaN,NaN,NaN,120
233,2021,4427353.0,3882111.78,120.64,119.58,106.40,876.8,7.437168,35.181260,2.140184e+09,...,20.772947,0.120518,1022.0,2712000.0,888000.0,5339000.0,NaN,NaN,NaN,180
259,2021,34684.0,30644.69,107.01,107.96,109.92,883.5,NaN,NaN,NaN,...,20.289856,0.121804,NaN,NaN,6700.0,134000.0,NaN,NaN,NaN,178


In [5]:
# WGI_World Governance Index
wbgapi.db = 3
series = pd.DataFrame(
    [item for item in wbgapi.series.info().items if item["id"].endswith("EST")]
)["id"].to_list()
wgi = wbgapi.data.DataFrame(series, "all", "all")
wgi = (
    wgi.rename(columns={f"YR{year}": year for year in range(1996, 2022)})
    .stack()
    .reset_index()
    .rename(columns={"level_2": "year", "economy": "country_code"})
    .pivot_table(
        columns="series",
        values=0,
        index=["country_code", "year"],
    )
)
wgi = wgi.loc[
    list(set(code_alpha3["Alpha-3 code"]) - {"ESH", "REU", "MYT"})
].sort_index()

wgi = wgi.rename(columns=lambda x: "WGI." + x)
wgi["WGI"] = wgi[
    ["WGI.CC.EST", "WGI.GE.EST", "WGI.PV.EST", "WGI.RQ.EST", "WGI.RL.EST", "WGI.VA.EST"]
].mean(axis=1)
WGI = (
    wgi.reset_index()
    .merge(code_alpha3, left_on="country_code", right_on="Alpha-3 code")
    .rename(
        columns={
            "year": "Year",
            "WGI.CC.EST": "cce",
            "WGI.GE.EST": "gee",
            "WGI.PV.EST": "pve",
            "WGI.RQ.EST": "rqe",
            "WGI.RL.EST": "rle",
            "WGI.VA.EST": "vae",
        }
    )
    .drop(columns=["country_code", "Alpha-3 code"])[
        ["Numeric", "Year", "cce", "gee", "pve", "rqe", "rle", "vae", "WGI"]
    ]
)

In [6]:
financial = (
    pd.read_excel(
        route
        / "World Bank Global Financial Development Database"
        / "20220909-global-financial-development-database.xlsx",
        sheet_name="Data - August 2022",
        na_values="..",
    )
    .rename(columns={"iso3": "Alpha-3 code", "year": "Year"})
    .merge(code_alpha3, on="Alpha-3 code")
    .drop(columns=["iso2", "imfn", "country", "region", "income"])
)

In [7]:
unsd_methodology = pd.read_csv(
    route / "Country Code" / "UNSD — Methodology.csv", sep=";"
).merge(
    pd.read_csv(route / "Country Code" / "UNSD — Methodology.zh_CN.csv"),
    on=[
        "M49 Code",
        "Global Code",
        "Region Code",
        "Sub-region Code",
        "Intermediate Region Code",
        "ISO-alpha2 Code",
        "ISO-alpha3 Code",
        "Least Developed Countries (LDC)",
        "Land Locked Developing Countries (LLDC)",
        "Small Island Developing States (SIDS)",
    ],
    suffixes=["", " (zh_CN)"],
)  # .astype({"M49 Code": "int64"})
unsd_methodology["Least Developed Countries (LDC)"] = (
    unsd_methodology["Least Developed Countries (LDC)"] == "x"
)
unsd_methodology["Land Locked Developing Countries (LLDC)"] = (
    unsd_methodology["Land Locked Developing Countries (LLDC)"] == "x"
)
unsd_methodology["Small Island Developing States (SIDS)"] = (
    unsd_methodology["Small Island Developing States (SIDS)"] == "x"
)

In [8]:
global_ = (
    unsd_methodology[["Global Code", "Global Name", "Global Name (zh_CN)"]]
    .drop_duplicates()
    .rename(
        columns={
            "Global Code": "Code",
            "Global Name": "Name",
            "Global Name (zh_CN)": "Name (zh_CN)",
        }
    )
    .set_index("Code")
)
global_["Level"] = "Global"
global_["Parent Code"] = 0
regions = (
    unsd_methodology[["Region Code", "Region Name", "Region Name (zh_CN)"]]
    .dropna()
    .drop_duplicates()
    .astype({"Region Code": "int64"})
    .rename(
        columns={
            "Region Code": "Code",
            "Region Name": "Name",
            "Region Name (zh_CN)": "Name (zh_CN)",
        }
    )
    .set_index("Code")
)
regions["Level"] = "Region"
regions["Parent Code"] = 1
sub_regions = (
    unsd_methodology[
        ["Sub-region Code", "Sub-region Name", "Sub-region Name (zh_CN)", "Region Code"]
    ]
    .dropna()
    .drop_duplicates()
    .astype({"Sub-region Code": "int64"})
    .rename(
        columns={
            "Sub-region Code": "Code",
            "Sub-region Name": "Name",
            "Sub-region Name (zh_CN)": "Name (zh_CN)",
            "Region Code": "Parent Code",
        }
    )
    .set_index("Code")
)
sub_regions["Level"] = "Sub-region"
intermediate_regions = (
    unsd_methodology[
        [
            "Intermediate Region Code",
            "Intermediate Region Name",
            "Intermediate Region Name (zh_CN)",
            "Sub-region Code",
        ]
    ]
    .dropna()
    .drop_duplicates()
    .astype({"Intermediate Region Code": "int64"})
    .rename(
        columns={
            "Intermediate Region Code": "Code",
            "Intermediate Region Name": "Name",
            "Intermediate Region Name (zh_CN)": "Name (zh_CN)",
            "Sub-region Code": "Parent Code",
        }
    )
    .set_index("Code")
)
intermediate_regions["Level"] = "Intermediate Region"
country_or_area = (
    unsd_methodology[
        [
            "M49 Code",
            "Country or Area",
            "Country or Area (zh_CN)",
            "Intermediate Region Code",
            "Sub-region Code",
            "Region Code",
            "Global Code",
        ]
    ]
    .rename(
        columns={
            "M49 Code": "Code",
            "Country or Area": "Name",
            "Intermediate Region Code": "Parent Code",
            "Country or Area (zh_CN)": "Name (zh_CN)",
        }
    )
    .set_index("Code")
)
country_or_area["Level"] = "Country or Area"
country_or_area.loc[
    country_or_area["Parent Code"].isna(), "Parent Code"
] = country_or_area.loc[country_or_area["Parent Code"].isna(), "Sub-region Code"]
country_or_area.loc[
    country_or_area["Parent Code"].isna(), "Parent Code"
] = country_or_area.loc[country_or_area["Parent Code"].isna(), "Region Code"]
country_or_area.loc[
    country_or_area["Parent Code"].isna(), "Parent Code"
] = country_or_area.loc[country_or_area["Parent Code"].isna(), "Global Code"]
country_or_area = (
    country_or_area.astype({"Parent Code": "int64"})
    .drop(columns=["Sub-region Code", "Region Code", "Global Code"])
    .sort_index()
)
all_regions = pd.concat(
    [global_, regions, sub_regions, intermediate_regions, country_or_area]
).astype({"Parent Code": "int64"})
all_regions.loc[158] = {
    "Name": "Taiwan Province of China",
    "Name (zh_CN)": "中国台湾省",
    "Level": "Country or Area",
    "Parent Code": 30,
}
all_regions = all_regions.sort_index()

In [9]:
greenfield_FDI = (
    pd.read_excel(route / "UNCTAD" / "wir2023_tab14.xlsx", skiprows=2, skipfooter=8)
    .melt(
        id_vars="Region/economy",
        var_name="Year",
        value_name="Value of announced greenfield FDI projects, by destination (Millions of dollars)",
    )
    .fillna(0)
)
rename_map = dict(
    (
        ("United States", "United States of America"),
        ("Netherlands", "Netherlands (Kingdom of the)"),
        ("Moldova, Republic of", "Republic of Moldova"),
        ("United Kingdom", "United Kingdom of Great Britain and Northern Ireland"),
        ("Korea, Republic of", "Republic of Korea"),
        ("Congo, Democratic Republic of", "Democratic Republic of the Congo"),
        ("São Tomé and Principe", "Sao Tome and Principe"),
        ("Côte d'Ivoire", "Côte d’Ivoire"),
        ("Hong Kong, China", "China, Hong Kong Special Administrative Region"),
        (
            "Korea, Democratic People's Republic of",
            "Democratic People's Republic of Korea",
        ),
        ("Macao, China", "China, Macao Special Administrative Region"),
        ("Iran, Islamic Republic of", "Iran (Islamic Republic of)"),
        ("Bolivia, Plurinational State of", "Bolivia (Plurinational State of)"),
        ("Venezuela, Bolivarian Republic of", "Venezuela (Bolivarian Republic of)"),
        ("Micronesia, Federated States of", "Micronesia (Federated States of)"),
        ("Other Africa", "Sub-Saharan Africa"),
    )
)
greenfield_FDI = greenfield_FDI.replace({"Region/economy": rename_map})
greenfield_FDI = greenfield_FDI.merge(
    all_regions.query("Level == 'Country or Area'")
    .reset_index()
    .rename(columns={"Code": "Numeric"}),
    left_on="Region/economy",
    right_on="Name",
).set_index(["Numeric", "Year"])[
    ["Value of announced greenfield FDI projects, by destination (Millions of dollars)"]
]

In [10]:
greenfield_FDI_num = (
    pd.read_excel(route / "UNCTAD" / "wir2023_tab17.xlsx", skiprows=2, skipfooter=8)
    .melt(
        id_vars="Region/economy",
        var_name="Year",
        value_name="Number of announced greenfield FDI projects, by destination",
    )
    .fillna(0)
    .astype({"Number of announced greenfield FDI projects, by destination": "int64"})
)
greenfield_FDI_num = greenfield_FDI_num.replace({"Region/economy": rename_map})
FDI_greenfield = (
    greenfield_FDI_num.merge(
        all_regions.query("Level == 'Country or Area'")
        .reset_index()
        .rename(columns={"Code": "Numeric"}),
        left_on="Region/economy",
        right_on="Name",
    )
    .set_index(["Numeric", "Year"])
    .merge(greenfield_FDI, left_index=True, right_index=True)[
        [
            "Number of announced greenfield FDI projects, by destination",
            "Value of announced greenfield FDI projects, by destination (Millions of dollars)",
        ]
    ]
)
FDI_greenfield["greenfdistock"] = (
    FDI_greenfield[
        "Value of announced greenfield FDI projects, by destination (Millions of dollars)"
    ]
    .sort_index()
    .groupby("Numeric")
    .cumsum()
)

In [11]:
merger_FDI_value = (
    pd.read_excel(route / "UNCTAD" / "wir2023_tab05.xlsx", skiprows=2, skipfooter=8)
    .melt(
        id_vars="     Region/economy",
        var_name="Year",
        value_name="Value of net cross-border M&As",
    )
    .fillna(0)
)

merger_FDI_value = merger_FDI_value.replace({"     Region/economy": rename_map})
merger_FDI_value = merger_FDI_value.merge(
    all_regions.query("Level == 'Country or Area'")
    .reset_index()
    .rename(columns={"Code": "Numeric"}),
    left_on="     Region/economy",
    right_on="Name",
).set_index(["Numeric", "Year"])[["Value of net cross-border M&As"]]


merger_FDI = (
    pd.read_excel(route / "UNCTAD" / "wir2023_tab07.xlsx", skiprows=2, skipfooter=8)
    .rename({"     Region/economy": "Region/economy"}, axis=1)
    .melt(
        id_vars="Region/economy",
        var_name="Year",
        value_name="Number of net cross-border M&As by region/economy of seller",
    )
    .astype({"Year": "int64"})
    .merge(
        all_regions.reset_index().rename(  # .query("Level == 'Country or Area'")
            columns={"Code": "Numeric"}
        ),
        left_on="Region/economy",
        right_on="Name",
    )
    .set_index(["Numeric", "Year"])[
        ["Number of net cross-border M&As by region/economy of seller"]
    ]
    .astype({"Number of net cross-border M&As by region/economy of seller": "float64"})
    .merge(merger_FDI_value, left_index=True, right_index=True)
)
merger_FDI["mafdistock"] = (
    merger_FDI["Value of net cross-border M&As"]
    .sort_index()
    .groupby("Numeric")
    .cumsum()
)

In [12]:
FDI_inflows = (
    pd.read_excel(route / "UNCTAD" / "wir2023_tab01.xlsx", skiprows=2, skipfooter=8)
    .melt(
        id_vars="Region/economy",
        var_name="Year",
        value_name="FDI inflows (Millions of dollars)",
    )
    .astype({"Year": "int64"})
    .replace({"Region/economy": rename_map})
    .merge(
        all_regions.reset_index().rename(  # .query("Level == 'Country or Area'")
            columns={"Code": "Numeric"}
        ),
        left_on="Region/economy",
        right_on="Name",
    )
    .set_index(["Numeric", "Year"])[["FDI inflows (Millions of dollars)"]]
    .astype({"FDI inflows (Millions of dollars)": "float64"})
)

In [13]:
FDI_instock = (
    pd.read_excel(route / "UNCTAD" / "wir2023_tab03.xlsx", skiprows=2, skipfooter=8)
    .melt(
        id_vars="Region/economy",
        var_name="Year",
        value_name="FDI instock (Millions of dollars)",
    )
    .astype({"Year": "int64"})
    .merge(
        all_regions.reset_index().rename(  # .query("Level == 'Country or Area'")
            columns={"Code": "Numeric"}
        ),
        left_on="Region/economy",
        right_on="Name",
    )
    .set_index(["Numeric", "Year"])[["FDI instock (Millions of dollars)"]]
    .astype({"FDI instock (Millions of dollars)": "float64"})
)

In [14]:
FDI = pd.concat(
    [FDI_inflows, FDI_instock, FDI_greenfield, merger_FDI], join="outer", axis=1
).reset_index()

In [15]:
# UNCTAD FDI stock
code_UNCTAD = countrycode[["Numeric", "UNCTADcountryname"]]

FDIstock = pd.read_excel(
    route / "UNCTAD" / "us_fdiflowsstock_48173467642114.xlsx",
    sheet_name="MySheet",
    header=4,
    na_values=["..", "_", "***"],
).rename(columns={"YEAR": "UNCTADcountryname"})

FDIstock["UNCTADcountryname"] = FDIstock["UNCTADcountryname"].str.replace("  ", "")

FDIstock = (
    FDIstock.merge(code_UNCTAD, on=["UNCTADcountryname"])
    .drop(columns=["UNCTADcountryname"])
    .melt(id_vars=("Numeric"))
    .rename(columns={"variable": "Year", "value": "FDIstock"})
    .astype({"Year": "int64", "FDIstock": "float64"})
)

In [16]:
###UNCTAD frontier technology readiness index
technology = (
    pd.read_excel(
        route / "UNCTAD" / "us_ftri_10734489584557.xlsx",
        sheet_name="MySheet",
        header=4,
        na_values=["..", "_", "***"],
    )
    .rename(columns={"Unnamed: 0": "UNCTADcountryname"})
    .drop([0])
    .melt(id_vars=["UNCTADcountryname", "YEAR"])
    .rename(columns={"variable": "Year"})
    .pivot_table(
        columns="YEAR",
        values="value",
        index=["UNCTADcountryname", "Year"],
    )
    .reset_index()
    .merge(code_UNCTAD, on=["UNCTADcountryname"])
    .drop(columns=["UNCTADcountryname"])
)
technology["Year"] = technology["Year"].astype(float)
technology["Numeric"] = technology["Numeric"].astype(float)

In [17]:
# Chinese OFDI
stock = (
    pd.read_excel(
        route / "Chinese OFDI" / "FDI.xlsx", sheet_name="Africa stock", na_values=".."
    )
    .melt(id_vars="国家（地区）")
    .rename(columns={"variable": "Year", "value": "stock", "国家（地区）": "CountryName_CN"})
)

flow = (
    pd.read_excel(
        route / "Chinese OFDI" / "FDI.xlsx", sheet_name="Africa flow", na_values=".."
    )
    .melt(id_vars="国家（地区）")
    .rename(columns={"variable": "Year", "value": "flow", "国家（地区）": "CountryName_CN"})
)

Chinafdi = (
    pd.merge(stock, flow, how="left", on=["CountryName_CN", "Year"])
    .merge(code_CN, on=["CountryName_CN"])[["Year", "stock", "flow", "Numeric"]]
    .astype({"Year": "int64"})
)

In [18]:
# Chinese Import and Export Data

# Chinese Import from Africa
im01 = pd.read_excel(
    route / "Chinese Import and Export Data" / "Trade between China and Africa.xlsx",
    sheet_name="进口额",
).drop(columns=["统计年鉴名"])

importfrom_Afri = (
    im01.merge(code_CN, how="right", on=["CountryName_CN"])
    .drop(columns=["CountryName_CN"])
    .melt(id_vars="Numeric")
    .rename(columns={"variable": "Year", "value": "importfrom_Afri"})
    .astype({"Year": "int64"})
)

# Chinese Export to Africa
ex01 = pd.read_excel(
    route / "Chinese Import and Export Data" / "Trade between China and Africa.xlsx",
    sheet_name="出口额",
).drop(columns=["统计年鉴名"])
exportto_Afri = (
    ex01.merge(code_CN, how="right", on=["CountryName_CN"])
    .drop(columns=["CountryName_CN"])
    .melt(id_vars=("Numeric"))
    .rename(columns={"variable": "Year", "value": "exportto_Afri"})
    .astype({"Year": "int64"})
)

# Chinese Trade with Africa
im_export01 = pd.read_excel(
    route / "Chinese Import and Export Data" / "Trade between China and Africa.xlsx",
    sheet_name="进出口总额",
).drop(columns=["统计年鉴名"])

im_export = (
    im_export01.merge(code_CN, how="right", on=["CountryName_CN"])
    .drop(columns=["CountryName_CN"])
    .melt(id_vars="Numeric")
    .rename(columns={"variable": "Year", "value": "im_ex"})
    .astype({"Year": "int64"})
)


trade = importfrom_Afri.merge(exportto_Afri, how="right", on=["Numeric", "Year"]).merge(
    im_export, how="right", on=["Numeric", "Year"]
)

In [19]:
# Country Classification and Geographic Distance

# World Bank Income Group
incomegroup = pd.read_excel(
    route / "Country Classification" / "World Bank Classification_Income Group.xlsx",
    sheet_name="Country Analytical History",
    na_values="..",
    header=10,
)
incomegroup = (
    incomegroup[["Alpha-3 code", *range(1996, 2023, 1)]]
    .merge(code_alpha3, how="right", on=["Alpha-3 code"])
    .drop(columns=["Alpha-3 code"])
    .melt(id_vars=("Numeric"))
    .rename(columns={"variable": "Year", "value": "incomegroup"})
)

# United Nations_Geometry_Natural Resources
dfgeo = pd.read_excel(
    route
    / "Country Classification"
    / "UN Classification_Natural resources_Geography.xlsx",
    sheet_name="Sheet1",
    na_values="..",
    usecols="D, E, F, G, H,I, M, N",
)
incomegroup = pd.merge(incomegroup, dfgeo, how="left", on=["Numeric"])

###CEPII Distance
# Distance from China to African Countries
dis0 = pd.read_excel(
    route / "CEPII Distance Data" / "dist_cepii.xlsx",
    sheet_name="dist_cepii",
    na_values="..",
)
dis = (
    dis0.query('iso_o == "CHN" ')
    .rename(columns={"iso_d": "Alpha-3 code"})[
        ["Alpha-3 code", "dist", "distcap", "distw", "distwces"]
    ]
    .merge(code_alpha3, how="right", on=["Alpha-3 code"])
    .drop(columns=["Alpha-3 code"])
)
group_dis0 = incomegroup.merge(dis, how="left", on=["Numeric"]).astype(
    {"Year": "int64"}
)
###CERDI seadistance databases
seadistance = (
    pd.read_excel(
        route / "CERDI seadistance" / "CERDI-seadistance.xlsx",
        sheet_name="CERDIseadistance",
        na_values="..",
    )
    .query("iso2=='NLD'")
    .rename(columns={"iso1": "Alpha-3 code"})
    .merge(code_alpha3, how="right", on="Alpha-3 code")
    .drop(columns=["roaddistance", "short", "iso2", "Alpha-3 code"])
)
seadistance

group_dis1 = group_dis0.merge(seadistance, how="left", on=["Numeric"]).astype(
    {"Year": "int64"}
)

###

commodityprice = (
    pd.read_excel(
        route / "World Bank Commodity Price" / "CMO-Historical-Data-Annual.xlsx",
        sheet_name="Annual Prices (Real)",
        na_values="..",
        header=[6],
    )
    .rename(columns={"Unnamed: 0": "Year"})
    .query("Year>1960")
)

commoditypriceindex = (
    pd.read_excel(
        route / "World Bank Commodity Price" / "CMO-Historical-Data-Annual.xlsx",
        sheet_name="Annual Indices (Real)",
        na_values="..",
        header=[9],
    )
    .rename(columns={"Unnamed: 0": "Year"})
    .query("Year>1960")
)

group_dis = (
    group_dis1.merge(commodityprice, how="right", on=["Year"])
    .astype({"Year": "int64"})
    .dropna(subset=["Numeric"])
    .merge(commoditypriceindex, how="right", on=["Year"])
    .astype({"Year": "int64"})
    .dropna(subset=["Numeric"])
)

In [20]:
# FAO SDG data
import faostat

fao_fs_csv = route / "FAO Data" / "Food Security.csv"

if fao_fs_csv.exists():
    FAO_FS = pd.read_csv(fao_fs_csv)
else:
    # Get the Food security dataset code
    dataset = (
        faostat.list_datasets_df()
        .query("label.str.contains('Security')")
        .iloc[0]["code"]
    )
    item = faostat.get_items(dataset)["-- Prevalence of undernourishment (percent)"]
    element = faostat.get_elements(dataset)["Value"]
    data = faostat.get_data_df(dataset, pars={"elements": [element], "items": item})
    # data = data.query("Year.str.match('^\d{4}$')").astype({"Year": int})
    data["Value"] = data["Value"].str.replace("<", "").replace("", "nan").astype(float)
    data = data.merge(
        countrycode[["Numeric", "CountryName"]].replace(
            {
                "Cape Verde": "Cabo Verde",
                "Congo, Rep.": "Congo",
                "Congo, Dem. Rep.": "Democratic Republic of the Congo",
                "Egypt, Arab Rep.": "Egypt",
                "Gambia, The": "Gambia",
                "Swaziland": "Eswatini",
                "São Tomé and Principe": "Sao Tome and Principe",
                "Tanzania": "United Republic of Tanzania",
            }
        ),
        left_on="Area",
        right_on="CountryName",
    )
    data["Year"] = data["Year"].str[:4].astype(int)
    FAO_FS = data[["Numeric", "Year", "Value"]].rename(
        columns={"Value": "Prevalence of undernourishment (percent) (3-year average)"}
    )

In [21]:
# SDR
sdr = (
    pd.read_excel(
        route / "SDR Database" / "SDR-2022-Database.xlsx",
        sheet_name="Raw Data - Trend Indicators",
        na_values="..",
    )
    .rename(columns={"id": "Alpha-3 code"})
    .merge(code_alpha3, on=["Alpha-3 code"])
    .drop(columns=["Alpha-3 code"])
)

In [22]:
# EPI_Environmental Performance Index
def epi(path):
    data = pd.read_csv(path)
    epi_var = path.stem.split("_")[0]
    if "CXN.mry" in data.columns:
        data = data.drop("CXN.mry", axis=1)
    if "WST.mry" in data.columns:
        data = data.drop("WST.mry", axis=1)
    data = data[data["code"].isin(countrycode["Numeric"])].rename(
        columns={"{}.raw.{}".format(epi_var, i): str(i) for i in range(1700, 2046)}
    )
    data.index = data.code
    data = data.drop(["iso", "country", "code"], axis=1).stack()
    data = data.rename(epi_var)
    return data


df = pd.DataFrame({"Alpha-3 code": [], "Year": []})
for p in Path(
    route
    / "EPI Environmental Performance Index"
    / "epi2022rawdata"
    / "2022 EPI Raw Data"
).glob("*_raw_na.csv"):
    if p.stem.startswith("."):
        continue
    data = epi(p)
    data = data.reset_index().rename(columns={"level_1": "Year", "code": "Numeric"})
    df = df.merge(data, how="outer")
epivariables = pd.read_excel(
    route / "EPI Environmental Performance Index" / "EPIvariables.xlsx"
)
EPI = df
EPI["Year"] = EPI["Year"].apply(pd.to_numeric)
EPI = EPI.rename(
    columns=dict(
        (k["Abbreviation"], k["Variable"]) for _, k in epivariables.iterrows()
    ),
).drop(columns=["Alpha-3 code"])

In [23]:
from datetime import datetime

if (route / "UN SDG Data" / "SDG-2023-07-31.csv").exists():
    SDG_panel = pd.read_csv(route / "UN SDG Data" / "SDG-2023-07-31.csv")
else:
    wbgapi.db = 46
    SDG = wbgapi.data.DataFrame(
        "all", code_alpha3["Alpha-3 code"].iloc[:-3], range(1996, 2021)
    )
    SDG_panel = (
        SDG.rename(columns={f"YR{year}": year for year in range(1996, 2021)})
        .stack()
        .unstack(level=1)
        .rename(
            columns={
                item["id"]: item["value"] for item in wbgapi.series.info(db=46).items
            }
        )
    )
    SDG_panel.index.rename(["Alpha-3 code", "Year"], inplace=True)
    SDG_panel = (
        SDG_panel.reset_index()
        .merge(code_alpha3, on="Alpha-3 code")
        .drop(columns=["Alpha-3 code"])
        .set_index(["Numeric", "Year"])
    )
    SDG_panel.to_csv(route / "UN SDG Data" / f"SDG-{datetime.today().isoformat()}.csv")
    SDG_panel = SDG_panel.reset_index()

In [24]:
# ILO Wages
code_ILO = countrycode[["Numeric", "ILOcountryname"]]
wage = (
    pd.read_excel(
        route / "ILO International Labour Organization" / "EAR_4MMN_CUR_NB_A_EN.xlsx",
        sheet_name="EAR_4MMN_CUR_NB_A_EN",
        na_values="..",
        header=5,
    )
    .rename(columns={"Reference area": "ILOcountryname"})
    .merge(code_ILO, on=["ILOcountryname"])
    .drop(columns=["Source type", "Unnamed: 6", "ILOcountryname", "Local currency"])
    .rename(
        columns={
            "Time": "Year",
            "2017 PPP $": "wage(2017 PPP $)",
            "U.S. dollars": "wage(U.S. dollars)",
        }
    )
)

In [25]:
# EFI_Index of Economic Freedom
code_efi = countrycode[["Numeric", "EFIcountryname"]]

EFI = (
    pd.read_csv(route / "EFI Index of Economic Freedom" / "data.csv")
    .rename(
        columns={"Name": "EFIcountryname", "Index Year": "Year", "Overall Score": "EFI"}
    )
    .merge(code_efi, on=["EFIcountryname"])
    .drop(columns=["EFIcountryname"])
)

In [26]:
duplicated_series = ["INTL.4701-34-WP17-BDOLPPP.A"]
EIA_INTL = pd.read_json(
    route / "EIA Energy Information Administration" / "INTL.zip", lines=True
)
duplicated_series = ["INTL.4006-8-MMTCD.A", "INTL.4002-8-MMTCD.A"]
EIA_INTL.dropna(subset=["series_id", "geoset_id", "data"], inplace=True)
for i in range(11, 28):
    EIA_INTL.query(f"not series_id.str.contains('WP{i}')", inplace=True)
EIA_INTL.query(f"not series_id.str.contains('-OPSA-')", inplace=True)
EIA_INTL.query("geoset_id not in @duplicated_series", inplace=True)
former_countries_or_EIA_specific_regions = [
    "WLD",
    "XKS",
    "HITZ",
    "USIQ",
    "DEUW",
    "USOH",
    "NLDA",
    "CSK",
    "DDR",
    "WAK",
    "YUG",
    "SCG",
    "SUN",
]
EIA_INTL.query(
    "geography not in @former_countries_or_EIA_specific_regions", inplace=True
)
EIA_INTL.query("f == 'A'", inplace=True)
EIA_INTL = EIA_INTL.explode("data").reset_index(drop=True)
EIA_INTL[["Year", "value"]] = (
    pd.DataFrame(
        EIA_INTL["data"].tolist(), index=EIA_INTL.index, columns=["Year", "value"]
    )
    .replace({"value": {"--": None, "w": np.nan, "NA": np.nan, "ie": np.nan}})
    .astype(
        {
            "Year": "int64",
            "value": "float64",
        }
    )
)
EIA_INTL = (
    EIA_INTL.drop(columns=["data"])
    .query(f"not geography.str.contains('\+')")[
        ["name", "geography", "Year", "value", "units"]
    ]
    .replace(
        r"(?P<Variables>Crude oil, NGPL, and other liquids production|Total energy consumption from nuclear, renewables, and other|Total energy production from nuclear, renewables, and other|Solar, tide, wave, fuel cell electricity .+?|.+?), (?P<region>Gambia, The|.+), Annual",
        r"\g<Variables>",
        regex=True,
    )
)
EIA_INTL["Variables"] = EIA_INTL["name"] + " (" + EIA_INTL["units"] + ")"
EIA_INTL = (
    EIA_INTL.drop(["name", "units"], axis=1)
    .pivot(index=["geography", "Year"], columns="Variables", values="value")
    .reset_index()
    .merge(code_alpha3, left_on="geography", right_on="Alpha-3 code")
)

from isocodes import countries

for geography in EIA_INTL.dropna(subset=['geography']).query('not geography.str.contains("\+")')["geography"].unique():
    match geography:
        case "WLD":
            region = {"name": "World", "numeric": '001', "alpha_3": "WLD"}
        case "XKS":
            region = {"name": "Kosovo"}
        case "HITZ":
            region = {"name": "Hawaiian Trade Zone"}
        case "USIQ":
            region = {"name": "U.S. Pacific Island"}
        case "DEUW":
            region = {"name": "Germany, West"}
        case "USOH":
            region = {"name": "U.S. Territories"}
        case "NLDA":
            region = {"name": "Netherlands Antilles"}
        case _:
            region = countries.get(alpha_3=geography)# or former_countries.get(alpha_3=geography)
            if not region:
                print(geography)

In [27]:
# Global data lab (globaldatalab.org)
GDL = (
    pd.read_excel(route / "GDL Global Data Lab" / "GDL-Mean-years-schooling-data.xlsx")[
        ["ISO_Code", *range(1990, 2022)]
    ]
    .melt(id_vars="ISO_Code", var_name="Year", value_name="Mean years schooling")
    .astype({"Year": "int64"})
    .merge(code_alpha3, left_on="ISO_Code", right_on="Alpha-3 code")
    .drop(columns=["ISO_Code", "Alpha-3 code"])
)

In [28]:
# Our world in data
# Education Years

educationyears0 = pd.read_csv(
    route / "Ourworldindata" / "mean-years-of-schooling-1.csv", header=0
).rename(
    columns={
        "Code": "Alpha-3 code",
        "Average Total Years of Schooling for Adult Population "
        "(Lee-Lee (2016), Barro-Lee (2018) and UNDP (2018))": ""
        "Average Total Years of Schooling for Adult Population",
    }
)

educationyears = educationyears0.merge(code_alpha3, on=["Alpha-3 code"]).drop(
    columns=["Alpha-3 code", "Entity"]
)

# COVID19
COVID19_0 = (
    pd.read_csv(route / "Ourworldindata" / "owid-covid-data_death.csv", header=0)[
        [
            "iso_code",
            "date",
            "total_cases",
            "total_cases_per_million",
            "total_deaths",
            "total_deaths_per_million",
        ]
    ]
    .query('date=="2020-12-31"')
    .rename(
        columns={
            "iso_code": "Alpha-3 code",
        }
    )
)
COVID19_0["Year"] = 2020
COVID19_0 = COVID19_0[
    [
        "Alpha-3 code",
        "Year",
        "total_cases",
        "total_deaths",
        "total_cases_per_million",
        "total_deaths_per_million",
    ]
]
COVID19 = (
    COVID19_0.merge(code_alpha3, on=["Alpha-3 code"])
    .drop(columns=["Alpha-3 code"])
    .astype({"Year": "int64"})
)
incidence_of_malaria = (
    pd.read_csv(route / "Ourworldindata" / "incidence-of-malaria.csv", header=0)
    .rename(
        columns={
            "Code": "Alpha-3 code",
        }
    )
    .merge(code_alpha3, on=["Alpha-3 code"])
)

In [29]:
# UNU-WIDER GINI

if not (
    route / "UNU-WIDER World Income Inequality Database" / "wiidglobal_1.csv"
).exists():
    pd.read_excel(
        route / "UNU-WIDER World Income Inequality Database" / "wiidglobal_1.xlsx"
    ).to_csv(route / "UNU-WIDER World Income Inequality Database" / "wiidglobal_1.csv")

gini = (
    pd.read_csv(
        route / "UNU-WIDER World Income Inequality Database" / "wiidglobal_1.csv"
    )[["c3", "year", "gini"]]
    .rename(columns={"c3": "Alpha-3 code", "year": "Year"})
    .merge(code_alpha3, on=["Alpha-3 code"])[
        ["Year", "Numeric", "gini", "Alpha-3 code"]
    ]
)

In [30]:
# Uppsala Conflict Data Program
code_UCDP = countrycode[["UCDPcountryname", "Numeric"]]
war = (
    pd.read_csv(
        route
        / "Uppsala Conflict Data Program"
        / "ucdp-prio-acd-211-csv"
        / "ucdp-prio-acd-211.csv",
        na_values="-99",
    )[["location", "year", "intensity_level"]]
    .rename(
        columns={
            "location": "UCDPcountryname",
            "year": "Year",
            "intensity_level": "war_intensity",
        }
    )
    .merge(code_UCDP, on="UCDPcountryname")
    .set_index("UCDPcountryname")
    .groupby(["Numeric", "Year"])
    .sum()
    .reset_index()
)

In [31]:
## Corruption Perception Index
##2012-2021
cpi = (
    pd.read_excel(
        route / "Corruption Perception Index" / "CPI2021_GlobalResults&Trends.xlsx",
        sheet_name="CPI Timeseries 2012 - 2021",
        na_values="..",
        header=2,
    )
    .set_index(["ISO3"])
    .filter(regex="CPI", axis=1)
    .rename(columns=lambda x: x.replace("CPI score", ""))
    .rename(columns=lambda x: x.replace("CPI Score", ""))
    .reset_index()
    .melt(id_vars="ISO3")
    .rename(
        columns={
            "variable": "Year",
            "value": "corruption perception index",
            "ISO3": "Alpha-3 code",
        }
    )
    .astype({"Year": "int64"})
).merge(code_alpha3, on=["Alpha-3 code"])

In [32]:
###employment in different sectors
employment = (
    pd.read_excel(
        route / "Economic Transformation Database" / "etd-release2021.xlsx",
        sheet_name="Data",
        na_values="..",
        header=0,
    )
    .query('var == "EMP"')
    .rename(columns={"cnt": "Alpha-3 code", "year": "Year"})
    .merge(code_alpha3, on=["Alpha-3 code"])
)
employment = employment.drop(columns=["Alpha-3 code", "var", "country"])

In [33]:
###global competitiveness index
###for covid19, discontinued since 2020
gci = (
    pd.read_excel(
        route / "Global Competitiveness Report Data" / "全球竞争力2006-2018.xlsx",
        sheet_name="Data",
        na_values="..",
        header=[2],
    )
    .rename(columns={"Edition": "Year"})
    .drop([0])
    .query('Attribute == "Value"')
    .drop(
        columns=[
            "Placement",
            "Dataset",
            "GLOBAL ID",
            "Series code",
            "Series unindented",
            "Attribute",
        ]
    )
)
gci["Year"] = gci["Year"].str[5:].astype("int64")
gci.drop(list(gci.filter(regex="Unnamed:")), axis=1, inplace=True)
gci = (
    gci.set_index(["Year", "Series"])
    .stack()
    .unstack(level=1)
    .reset_index()
    .rename(columns={"level_1": "Alpha-3 code"})
    .merge(code_alpha3, on=["Alpha-3 code"])
)
gci = gci.drop(columns=["Alpha-3 code", float("nan")])

In [34]:
import re

dfs = []
for csv in Path(route / "IMF" / "IMF-CID").glob("*.csv"):
    i, *rest = csv.stem.split("_")
    i = int(i)
    category = "_".join(rest)
    if i in (
        0,  # summary
        1,  # not annually data
        8,  # bilateral trade
        10,  # bilateral trade
        13,  # need to be investigated
        16,  # future prediction
        17,  # future prediction
        26,  # only world data
        27,  # only world data
    ):
        continue
    df = pd.read_csv(csv)
    years = [int(col) for col in df.columns if re.match(r"\d{4}", col)]
    nunique = df.nunique()
    cols_to_drop = nunique[nunique <= 1].index
    df = (
        df.drop(cols_to_drop, axis=1)
        .rename(columns={str(year): year for year in years})
        .rename(columns={"ISO3": "Alpha-3 code"})
        .dropna(subset=["Alpha-3 code"])
    )
    df["Variable"] = df[
        [
            col
            for col in df.columns
            if col not in ("Country", "ISO2", "Alpha-3 code", *years)
        ]
    ].agg("|".join, axis=1)
    df = (
        df.melt(id_vars=["Alpha-3 code", "Variable"], value_vars=years, var_name="Year")
        .pivot(index=["Alpha-3 code", "Year"], columns="Variable", values="value")
        .reset_index()
        .astype({"Year": int})
        .set_index(["Alpha-3 code", "Year"])
    )
    dfs.append(df)
IMF_CID = (
    pd.concat(dfs, axis=1)
    .reset_index()
    .merge(code_alpha3, on="Alpha-3 code")
    .drop(columns=["Alpha-3 code"])
)

In [35]:
current_account = (
    pd.read_excel(
        route / "IMF" / "imf-dm-export-20230929.xls",
        sheet_name="BCA_NGDPD",
        na_values="no data",
    )
    .rename(
        columns={"Current account balance, percent of GDP (Percent of GDP)": "IMFName"}
    )
    .merge(countrycode[["IMFName", "Numeric", "Alpha-3 code"]], on=["IMFName"])
    .drop(columns=["IMFName", "Alpha-3 code"])
    .melt(id_vars=["Numeric"])
    .rename(
        columns={
            "variable": "Year",
            "value": "Current account balance, percent of GDP (Percent of GDP)(IMF)",
        }
    )
)

In [36]:
CDIS = pd.read_csv(route / "IMF" / "CDIS_12-07-2022 14-14-34-64_timeSeries.zip").drop("Unnamed: 20", axis=1)
regions = {
    "China, P.R.: Hong Kong": "FDI_HK",
    "China, P.R.: Macao": "FDI_MO",
    "China, P.R.: Mainland": "FDI_CN",
    "Taiwan Province of China": "FDI_TW",
    "United States": "FDI_US",
    "United Kingdom": "FDI_GB",
    "Netherlands, The": "FDI_NL",
    "France": "FDI_FR",
}
CDIS.melt(id_vars=["Country Name", "Country Code", "Indicator Name", "Indicator Code", "Counterpart Country Name", "Counterpart Country Code", "Attribute"], var_name="Year", value_name="ODIP").query("`Indicator Name` == 'Outward Direct Investment Positions, US Dollars' and Attribute == 'Value' and `Country Name` in @regions").replace({"Country Name": regions}).pivot_table(index=["Counterpart Country Name", "Year"], columns="Country Name", values="ODIP")
name_map = {
    row[1]: row[2]
    for row in countrycode[["IMF_CDIS_Name", "Numeric"]].itertuples()
}
IMF_CDIS = CDIS.melt(id_vars=["Country Name", "Country Code", "Indicator Name", "Indicator Code", "Counterpart Country Name", "Counterpart Country Code", "Attribute"], var_name="Year", value_name="ODIP").query("`Indicator Name` == 'Outward Direct Investment Positions, US Dollars' and Attribute == 'Value' and `Country Name` in @regions").replace({"Country Name": regions}).pivot_table(index=["Counterpart Country Name", "Year"], columns="Country Name", values="ODIP").reset_index().replace({"Counterpart Country Name": name_map}).query("`Counterpart Country Name` in @countrycode['Numeric']").rename(columns={"Counterpart Country Name": "Numeric"}).astype({"Year": int}).set_index(["Numeric", "Year"])

In [37]:
FDI_US = pd.DataFrame(columns=["Country", "Year", "FDI_USBEA"])
for year in range(1989, 2023):
    if year in range(1989, 2000):
        io = route / "USBEA US Direct Investment Abroad" / "USDIA Detailed Country by Selected Industry 1989-1999.xls"
    elif year in range(2000, 2010):
        io = route / "USBEA US Direct Investment Abroad" / "USDIA Detailed Country by Selected Industry 2000-2009.xls"
    elif year in range(2010, 2020):
        io = route / "USBEA US Direct Investment Abroad" / "usdia-detailedcountrybyselectedindustry-2010-2019.xlsx"
    elif year in range(2020, 2023):
        io = route / "USBEA US Direct Investment Abroad" / "usdia-detailedcountrybyselectedindustry-2020-2022.xlsx"
    else:
        raise ValueError(f"Year {year} is not included.")
    skiprows = 4
    if year in range(1999, 2003):
        skiprows = 5
    year_data = pd.read_excel(io, sheet_name=str(year), skiprows=skiprows).iloc[:, :2]
    year_data.columns = ["Country", "FDI_USBEA"]
    year_data["Country"] = year_data["Country"].str.strip()
    year_data["Year"] = year
    FDI_US = pd.concat([FDI_US, year_data.dropna().replace({"(D)": float("nan"), "(*)": float("nan")}).astype({"FDI_USBEA": "float64"})])
name_map = {
    "Algeria": 12,
    "Angola": 24,
    "Benin": 204,
    "Botswana": 72,
    "Burundi": 108,
    "Cabo Verde": 132,
    "Cameroon": 120,
    "Central African Republic": 140,
    "Chad": 148,
    "Comoros": 174,
    "Congo": 178,
    "Congo (Brazzaville)": 178,
    "Congo (Kinshasa)": 180,
    "Cote D'Ivoire": 384,
    "Cote d'Ivoire": 384,
    "Djibouti": 262,
    "Egypt": 818,
    "Equatorial Guinea": 226,
    "Eritrea": 232,
    "Eswatini": 748,
    "Ethiopia": 231,
    "Gabon": 266,
    "Gambia": 270,
    "Ghana": 288,
    "Guinea": 324,
    "Guinea-Bissau": 624,
    "Kenya": 404,
    "Lesotho": 426,
    "Liberia": 430,
    "Libya": 434,
    "Madagascar": 450,
    "Malawi": 454,
    "Mali": 466,
    "Mauritania": 478,
    "Mauritius": 480,
    "Morocco": 504,
    "Morocco 2": 504,
    "Mozambique": 508,
    "Namibia": 516,
    "Niger": 562,
    "Nigeria": 566,
    "Rwanda": 646,
    "Sao Tome and Principe": 678,
    "Senegal": 686,
    "Seychelles": 690,
    "Sierra Leone": 694,
    "Somalia": 706,
    "South Africa": 710,
    "South Sudan": 728,
    "Sudan": 729,
    "Sudan 3": 729,
    "Tanzania": 834,
    "Togo": 768,
    "Tunisia": 788,
    "Uganda": 800,
    "Western Sahara": 732,
    "Zambia": 894,
    "Zimbabwe": 716,
}
FDI_USBEA = FDI_US.query("Country in @name_map").replace({"Country": name_map}).rename(columns={"Country": "Numeric"}).set_index(["Numeric", "Year"]).sort_index()
FDI_USBEA

FDI_USBEA
Numeric Year           
12      1989        6.0
        1990        1.0
        1991        6.0
        1992       52.0
        1993        NaN
...                 ...
894     2018       49.0
        2019      -22.0
        2020      -88.0
        2021     -117.0
        2022     -157.0

[1520 rows x 1 columns]

In [38]:
from functools import reduce

# Merge all the data
dfs = {
    "WDI": WDI,
    "WGI": WGI,
    "financial": financial,
    "China FDI": Chinafdi,
    "Sino-Africa Trade": trade,
    "FDI": FDI,
    "FDI stock": FDIstock,
    "techonology": technology,
    "Group distance": group_dis,
    "EPI": EPI,
    "Malaria": incidence_of_malaria,
    "SDR": sdr,
    "SDG": SDG_panel,
    "FAO_FS": FAO_FS,
    "wage": wage,
    "EFI": EFI,
    "GDL": GDL,
    "Education years": educationyears,
    "COVID19": COVID19,
    "GINI": gini,
    "War": war,
    "CPI": cpi.drop(columns=["Alpha-3 code"]),
    "Employment": employment,
    "GCI": gci,
    "EIA_INTL": EIA_INTL.drop(["geography", "Alpha-3 code"], axis=1),
    "IMF_CID": IMF_CID,
    "current_account":current_account,
    "IMF_CDIS": IMF_CDIS,
    "FDI_USBEA": FDI_USBEA,
}

df_final = reduce(
    lambda left, right: pd.merge(left, right, how="outer", on=("Year", "Numeric")),
    dfs.values(),
)
df_final = (
    df_final.merge(countrycode, how="left", on=["Numeric"], suffixes=("", "_DROP"))
    .filter(regex="^(?!.*_DROP)")
    .fillna({"wardummy": 0})
)


df_final.sort_index().to_csv(route / "Data cleaning" / "df_final.csv", index=False)

In [39]:
df_final[["Year", "Current account balance, percent of GDP (Percent of GDP)(IMF)"]].query("Year == 2021")

,Year,"Current account balance, percent of GDP (Percent of GDP)(IMF)"
25,2021,11.2
51,2021,-12.4
77,2021,-4.2
103,2021,-0.4
129,2021,-0.5
...,...,...
9341,2021,NaN
9374,2021,NaN
9407,2021,NaN
9425,2021,NaN


In [40]:
def variable_source_generator():
    for name, df in dfs.items():
        if name in ("Malaria", "Education years"):
            source = "Our world in data"
        elif name in ("FDI", "FDI stock"):
            source = "UNCTAD"
        elif name == "wage":
            source = "ILO"
        elif name == "Group distance":
            source = "CEPII"
        elif name == "Employment":
            source = "Economic Transformation Database"
        elif name == "War":
            source = "Uppsala Conflict Data Program"
        else:
            source = name
        for col in df:
            if col in ("Numeric", "Year"):
                continue
            yield {"variable": col, "source": source}


pd.DataFrame(variable_source_generator()).to_excel(
    route / "Data cleaning" / "indicatorx.xlsx", index=False
)

In [41]:
roi = list(set(code_alpha3["Alpha-3 code"]) - {"REU", "MYT", "SOM"})
df_final.query("Year >= 2005 and Year <= 2020 and `Alpha-3 code` in @roi")[
    "Year"
].unique()

array([2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015,
       2016, 2017, 2018, 2019, 2020], dtype=object)

In [42]:
d = pd.DataFrame(df_final["Year"].unique()).rename(columns={0: "year"})
d.query("year>2005")

,year
10,2006
11,2007
12,2008
13,2009
14,2010
15,2011
16,2012
17,2013
18,2014
19,2015


In [43]:
df_final.query("Year == 2021")

,Year,"Agricultural machinery, tractors",Fertilizer consumption (% of fertilizer production),Fertilizer consumption (kilograms per hectare of arable land),Agricultural land (sq. km),Agricultural land (% of land area),Arable land (hectares),Arable land (hectares per person),Arable land (% of land area),Land under cereal production (hectares),...,ILOcountryname,EFIcountryname,Map,EIAcountryname,UNCTADcountryname,UCDPcountryname,FAOStatAreaName,IMFName,IMF_CDIS_Name,Alpha-2 code
25,2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3052269.0,...,Angola,Angola,Angola,Angola,Angola,Angola,Angola,Angola,Angola,AO
51,2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,309009.0,...,Burundi,Burundi,Burundi,Burundi,Burundi,Burundi,Burundi,Burundi,Burundi,BI
77,2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1639128.0,...,Benin,Benin,Benin,Benin,Benin,Benin,Benin,Benin,Benin,BJ
103,2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3761051.0,...,Burkina Faso,Burkina Faso,Burkina Faso,Burkina Faso,Burkina Faso,Burkina Faso,Burkina Faso,Burkina Faso,Burkina Faso,BF
129,2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,121752.0,...,Botswana,Botswana,Botswana,Botswana,Botswana,Botswana,Botswana,Botswana,Botswana,BW
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9341,2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9374,2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9407,2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9425,2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Western Sahara,Western Sahara,Western Sahara,Western Sahara,Sahrawi Arab Democratic Republic (Western Sahara),NaN,NaN,Western Sahara,EH
